In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import cv2
from models import ResNet18

In [3]:
model = ResNet18(num_classes=10)
model1 = torch.load('/home/stu6/2024PRML_Project2/TaskA/2024-06-03-18:20:35/best_model.pt')